In [1]:

import numpy as np 
import pandas as pd 
import seaborn as sb
import xgboost as xgb


from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

In [4]:
#explore the data


train = pd.read_csv("/home/amal/Téléchargements/train_50.csv")
#print(train.head())

test = pd.read_csv("/home/amal/Téléchargements/test_50.csv")

In [5]:
#sum features
featuresSum = train.sum()
#print(featuresSum)
#print(type(train))
#plot the sum of features in desceding order
featuresSum.drop(['target', 'id']).order().plot(kind='barh', figsize=(8,16))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: FutureWarning: order is deprecated, use sort_values(...)


In [6]:
#Clases are String like "Class_1" => we map them as numbers from 0-8 (9 classes)

class_range = range(1, 10)
class_dict = {}

for n in class_range:
    class_dict['Class_{}'.format(n)] = n-1

#print(class_dict)
#print(train.head())
train['target'] = train['target'].map(class_dict)
#print(train['target'].head())

#next we plot the count for each class unsing seaborn libary
sb.countplot(x='target', data= train)

In [7]:
#next we create training and testing sets
X_train = train.drop(["id", "target"], axis=1)
Y_train = train["target"].copy()
X_test = test.drop("id", axis = 1).copy()

In [13]:
# we use the training and test sets to make a prediction based on Boosted Trees => library XgBoost
# http://xgboost.readthedocs.io/en/latest/model.html

params = {"objective": "multi:softprob", "eval_metric":"mlogloss", "num_class": 9}

T_train_xgb = xgb.DMatrix(X_train, Y_train)
X_test_xgb  = xgb.DMatrix(X_test)

#number of iterations - initial 20, 
gbm = xgb.train(params, T_train_xgb, 50)
Y_pred = gbm.predict(X_test_xgb)
print(Y_pred)

[[  3.09625349e-04   5.26831388e-01   3.93590331e-01 ...,   1.01200612e-02
    1.39244401e-03   1.31381341e-04]
 [  8.51289893e-04   9.27847717e-03   2.20594043e-03 ...,   1.58328272e-03
    1.89369693e-01   9.39092482e-04]
 [  8.12630897e-05   2.97565421e-04   1.23004473e-04 ...,   1.50788022e-04
    3.60993843e-04   8.53675738e-05]
 ..., 
 [  3.90368339e-04   5.03351092e-01   2.47188538e-01 ...,   6.10784674e-03
    2.07993013e-04   2.16426561e-04]
 [  1.20221834e-04   2.48132199e-01   8.16784650e-02 ...,   3.10692866e-03
    1.03520440e-04   1.29586289e-04]
 [  3.58198141e-03   3.11427623e-01   4.54801291e-01 ...,   1.50145888e-01
    1.47303555e-03   6.85389154e-04]]


In [12]:
#Make a submission based on model "sample_submission.csv"

submission = pd.DataFrame({ "id": test["id"]})

i = 0

for num in class_range:
    col_name = str("Class_{}".format(num))
    submission[col_name] = Y_pred[:,i]
    i = i + 1
    
submission.to_csv('/home/amal/Bureau/xg_reduced.csv', index=False)